# Psi4 Lab 01: Introduction


## Import psi4, set the total memory, and output file


In [1]:
import psi4

psi4.set_memory('1 GB')
psi4.core.set_output_file('output.dat', False)

In [3]:
import psi4

hatom = psi4.geometry("""
0 2
H
""")

psi4.set_options({'reference': 'uhf'})
eng = psi4.energy('hf/sto-3g', molecule=hatom)
print(eng)

-0.4665818495572755


In [6]:
charge = 0
multp = 1
rHH = 1.0 # Ångstrom
geom = f'{charge} {multp}\nH\nH 1 {rHH:.6f}' # we use formatted strings
print(geom)

0 1
H
H 1 1.000000


In [7]:
mol = psi4.geometry(geom)

In [10]:
print(f"Number of atoms = {mol.natom()}")
print(f"Nuclear repulsion energy = {mol.nuclear_repulsion_energy()}")

Number of atoms = 2
Nuclear repulsion energy = 0.5291772106699998


In [9]:
basis = 'cc-pVDZ'
reference = 'rhf'

psi4.core.clean()

psi4.set_options({'basis': basis,'scf_type': 'pk', 'reference' : reference})

# pipe output to the file output.dat
psi4.core.set_output_file('output.dat', False)

# run scf and return the energy and a wavefunction object (will work only if pass return_wfn=True)
E_scf, wfn = psi4.energy('scf', return_wfn=True)

In [11]:
print(f'SCF Energy: {E_scf}')

SCF Energy: -1.1001537647935047


In [13]:
print(f"Number of orbitals = {wfn.nmo()}")
print(f"Number of alpha electrons = {wfn.nalpha()}")
print(f"Number of irreducible representations (irreps) = {wfn.nirrep()}")
nirrep = wfn.nirrep()

Number of orbitals = 10
Number of alpha electrons = 1
Number of irreducible representations (irreps) = 8


In [ ]:
co_cart = psi4.geometry("""
0 1
C 0.000000 0.000000 0.000000
O 0.000000 0.000000 1.128323
""")

Let's check the geometry with the `to_string` function

In [ ]:
print(co_cart.to_string(dtype='xyz'))

## Z-matrix coordinates

Another way to provide a geometry is via the Z-matrix format. It is more natural to setup some geometries using the Z-matrix format and so it has some advantages. However, for larger molecules Cartesian coordinates are usually better. The Z-matrix format is best learned via a series of examples. 
For a simple diatomic A-B the Z-matrix is simply
```
A
B 1 <distance A-B>
```

For example, for CO we specify the Z-matrix in the following way (note how the result is equivalent to that of the XYZ geometry)

In [ ]:
co_zmat = psi4.geometry("""
0 1
C
O 1 1.128323
""")

print(co_zmat.to_string(dtype='xyz'))

In this Z-matrix:
- `C` indicates that the first atom is a carbon.
- `O 1 1.128323` indicates that the second atom is an oxygen, and that this atom is 1.128323 Å away from the first atom.

Now we can go ahead and optimize the geometry

In [ ]:
psi4.optimize('scf/cc-pvdz', molecule=co_zmat)

In [ ]:
print(co_zmat.to_string(dtype='xyz'))

## Z-matrix for triatomic molecules

Let's now move to a triatomic molecule, water. In the general case of a Z-matrix for the A-B-C molecule there is one additional line for the third atom (C). To give a geometry we have to specify: the atom X from which we will measure the distance of C, the distance C-X, a second atom Y which will be used to specify the bond angle C-X-Y, and the bond angle C-X-Y
```
A
B 1 <distance A-B>
C <connected to X> <distance C-X> <angle to Y> <angle C-X-Y (degrees)>
```

For the water molecule we can number the atoms in the following way

<img src="./water_zmat.pdf" alt="h2 molecule" width="200"/>

If we use a bond distance of 1 Å and an angle of 104.5 degrees then the Z-matrix is given by

In [ ]:
h2o_zmat = psi4.geometry("""
0 1
O
H 1 1.0
H 1 1.0 2 104.5
""")

print(h2o_zmat.to_string(dtype='xyz'))

Let's run a Hartree–Fock computation using the cc-pVDZ basis and optimize the geometry

In [ ]:
scf_e = psi4.optimize('scf/cc-pvdz', molecule=h2o_zmat)

In [ ]:
print(f'Energy of water: {scf_e}')
print(h2o_zmat.to_string(dtype='xyz'))

### Exercise

1. Build a zmat for a **linear** water molecule and optimize its geometry.
1. Verify that the geometry is still linear. Then compute the energy difference between the bent and linear geometries, converting the energy difference to kcal/mol.
1. **Challenge** Run a computation that will tell you if the linear molecule is a minimum or a transition state.

### Solution

In [ ]:
h2o_zmat_lin = psi4.geometry("""
0 1
O
H 1 1.0
H 1 1.0 2 180.0
""")

print(h2o_zmat_lin.to_string(dtype='xyz'))

In [ ]:
scf_e_lin = psi4.optimize('scf/cc-pvdz', molecule=h2o_zmat_lin)

In [ ]:
print(h2o_zmat_lin.to_string(dtype='xyz'))

In [ ]:
print(f'Barrier to linearity {627.51*(scf_e_lin-scf_e):.2f} kcal/mol')

In [ ]:
h2o_zmat_bent2 = psi4.geometry("""
0 1
O
H 1 1.0
H 1 1.0 2 170.0
""")

print(h2o_zmat_bent2.to_string(dtype='xyz'))
scf_e_bent2 = psi4.optimize('scf/cc-pvdz', molecule=h2o_zmat_bent2)
print(h2o_zmat_bent2.to_string(dtype='xyz'))

In [ ]:
print(scf_e_bent2)

## Z-matrix for polyatomic molecules

Molecules with more than three atoms extend beyond a 2D plane, and so we need to specify dihedreal angles. For example, in the case of hydrogen peroxide (H<sub>2</sub>O<sub>2</sub>) we can number the atoms as

<img src="./h2o2_zmat.pdf" alt="h2 molecule" width="400"/>

Here is a useful chart when talking about dihedral angles (source wikipedia)

<img src="./synantipericlinal.png" alt="h2 molecule" width="200"/>


The Z-matrix for hydrogen peroxide is 
```
O
O 1 1.4
H 2 1.0 1 120.0
H 1 1.0 2 120.0 3 60.0
```
where in the fourth line we specify that atom H4 has a distance of 1.0 Å from atom O1, it forms a 120 degree angle with atoms O1-O2, and the dihedral angle H4-O1-O2-H3 is 60 degress. The corresponding psi4 input is:

In [ ]:
h2o2_zmat = psi4.geometry("""
0 1
O
O 1 1.4
H 2 1.0 1 120.0
H 1 1.0 2 120.0 3 60.0
""")

print(h2o2_zmat.to_string(dtype='xyz'))

Note that it possible to indicate bond legths, angles, and dihedrals with symbols and assign these values after the Z-matrix after inserting an empty line

In [ ]:
h2o2_zmat2 = psi4.geometry("""
0 1
O
O 1 ROO
H 2 ROH 1 A
H 1 ROH 2 A 3 D

ROH = 1.000
ROO = 1.400
A = 120.0
D = 120.0
""")

print(h2o2_zmat2.to_string(dtype='xyz'))

This feature is convenient if we want to scan a geometry, for example, change the dihedral angle

In [ ]:
h2o2_zmat2 = psi4.geometry("""
0 1
O
O 1 ROO
H 2 ROH 1 A
H 1 ROH 2 A 3 D

ROH = 1.000
ROO = 1.400
A = 120.0
D = 120.0
""")

print(h2o2_zmat2.to_string(dtype='xyz'))

## Dummy atoms

Sometimes it is convenient to use dummy atoms when constructing a Z-matrix input. A dummy atom does not carry electrons or basis functions, and can be useful to specify an anchor atom with respect to which you can specify the molecular geometry.

Dummy atoms also are helpful to remove ambiguity in the Z-matrix. Except for triatomics, the Z-matrix format does not accept bond angles of 180 degrees. This is for example the case in acetylene. If we try to specify the geometry of acetylene without dummy atoms, psi4 gives an error since it cannot convert the Z-matrix to Cartesian structure

In [ ]:
c2h2_zmat = psi4.geometry("""
H
C 1 HC
C 2 CC 3 A
H 3 HC 2 A 1 D
HC = 1.08
CC = 1.2
A = 180.0
D = 0.0
""")

print(c2h2_zmat.to_string(dtype='xyz'))

To avoid this issue we need to introduce two dummy atoms arranged in the following way

<img src="./c2h2_zmat.pdf" alt="h2 molecule" width="400"/>

In [ ]:
c2h2_zmat = psi4.geometry("""
H
C 1 HC
X 2 CX 1 A1
C 2 CC 3 A1 1 D1
X 4 CX 2 A1 1 D1
H 4 HC 5 A1 2 D1

HC = 1.08
CX = 1.0
CC = 1.2
A1 = 90.0
D1 = 180.0
""")

print(c2h2_zmat.to_string(dtype='xyz'))

## Angstrom vs. Bohr

So far all inputs have assumed coordinates in units of Å. To specify the geometry in bohr, just add a line at the end of you geometry `units {unit}` where `spec` is one of `ang`, `angstrom`, `a.u.`, `au`, or `bohr`. The latter three specify atomic units

In [ ]:
h2o_zmat_bohr = psi4.geometry("""
0 1
O
H 1 1.89036
H 1 1.89036 2 104.5

units bohr
nocom
noreorient
""")

print(h2o_zmat_bohr.to_string(dtype='xyz'))

## Fancier options

There are several other options worth knowing that can be added at the end of the molecule block:
1. `noreorient` disable reorientation of the molecule.
1. `nocom` disable shifting the molecule to the center of mass.
1. `symmetry c1` run a computation ingoring the molecular symmetry.

A neat feature is that we can directly query the [pubchem database](http://www.psicode.org/psi4manual/master/psithonmol.html#sec-pubchem) and grab a geometry by specifying its name  or the CID number.

In [ ]:
pubchem = psi4.geometry("""
pubchem:benzene
""")

print(pubchem.to_string(dtype='orca'))

## Resources 

To read more about molecule and geometry specifications in psi4 check out the [manual page
](http://www.psicode.org/psi4manual/master/psithoninput.html#molecule-and-geometry-specification).